<a href="https://colab.research.google.com/github/DavinciB/child_grooming_detector/blob/main/src/SVM/1%20-%20SVM%20Data%20Prefiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
"""SVM Data Prefiltering

This series of notebook will be used for the SVM implementation for online 
grooming. We will re-implement the SVM suggested by Villatoro-Tello et al.
in the paper "A Two-step Approach for Effective Detection of Misbehaving
Users in Chats".
Prefiltering Training Data

First, we want to do the required pre-filtering of training data. Any
conversation meeting any of the following criteria are removed:

    conversations with only one participant
    conversations with each user having 6 or less messages
    conversations with long sequences of unrecognized characters"""

import xml.etree.ElementTree as ET
import datetime

"""train_data_path = "../../data/pan12-sexual-predator-identification-training-corpus-2012-05-01/"
test_data_path = "../../data/pan12-sexual-predator-identification-test-corpus-2012-05-21/"

training_xml = ET.parse(train_data_path + 'pan12-sexual-predator-identification-training-corpus-2012-05-01.xml')
root = training_xml.getroot()*/"""

train_data_path = r"/content/drive/MyDrive/online-grooming-detector-master/data/pan12-sexual-predator-identification-training-corpus-2012-05-01"
test_data_path = r"/content/drive/MyDrive/online-grooming-detector-master/data/pan12-sexual-predator-identification-test-corpus-2012-05-21"

training_xml = ET.parse(train_data_path + '/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml')
root = training_xml.getroot()        


In [14]:
"""Now root stores the xml file in a nested list, let's remove conversations 
meeting the first criteria: conversations with only one participant"""
conv_2_remove = []
authors = []
init_len = len(root)

for conversation in root:
    authors.clear()
    
# finding all unique authors in this conversation
    for message in conversation:
        author = message.find('author').text
        if author not in authors:
            authors.append(author)

    if (len(authors)) <= 1 and conversation.get('id') not in conv_2_remove:
        conv_2_remove.append(conversation.get('id'))

print("{} out of {} conversations should be removed".format(len(conv_2_remove), init_len))

12773 out of 66927 conversations should be removed


In [15]:
"""We have now removed 12773 conversations. Next we remove all conversations 
that meet the second criteria: conversations with each user having 5 or less
 messages"""
for conversation in root:
    if conversation.get('id') in conv_2_remove:
        continue
    authors = {}
    for message in conversation:
        author = message.find('author').text
        if author in authors:
            authors[author] = authors[author] + 1
        else:
            authors[author] = 1
    remove = True
    for author in authors:
        if authors[author] > 5:
            remove = False
    if remove is True and conversation.get('id') not in conv_2_remove:
        conv_2_remove.append(conversation.get('id'))

print("{} out of {} conversations should be removed".format(len(conv_2_remove), init_len))

51827 out of 66927 conversations should be removed


In [16]:
"""We have now removed 51827 conversations out of the original 66927.
Lastly we will remove any conversations with messages containing long
sequences of unrecognized characters"""

import re
for conversation in root:
    if conversation.get('id') in conv_2_remove:
        continue
    remove = False
    for message in conversation:
        text = message.find("text").text
        if text is None or len(text) < 20:
            continue
        match_str = re.findall("[\W_]", text)
        if len(match_str) / len(text) > 0.6:
            remove = True
            break

    if remove is True and conversation.get('id') not in conv_2_remove:
            conv_2_remove.append(conversation.get('id'))

print("{} out of {} conversations should be removed".format(len(conv_2_remove), init_len))

52224 out of 66927 conversations should be removed


In [17]:
"""We have now removed 52224 out of 66927 conversations.
Next we remove the conversations we want to remove from root itself
and write the new xml back into aother file."""
for conversation in root.findall('conversation'):
    if conversation.get('id') in conv_2_remove:
        root.remove(conversation)
print("The new root has a length of {}.".format(len(root)))


The new root has a length of 14703.


In [20]:
from xml.etree.ElementTree import ElementTree
tree = ElementTree(root)
tree.write(open(r'/content/drive/MyDrive/online-grooming-detector-master/data/svm_training_data/training_data.xml', 'wb'))
print("Filtered data written!")


Filtered data written!
